## Financial Crime - Fraud Detection

Your goal is to design an algorithm to identify fraudsters and take appropriate action. In this case, a fraudster is someone who signs up and uses Revolut to steal other people’s money from the outside into an account with us via top-ups. You are provided with the first 2 weeks of users’ ​transactions​ and info on ​users​ that is available from the start, at sign-up. Here, to streamline some of the decision-making, we make a few assumptions.

In [2]:
import pandas as pd

In [3]:
#Read files
fraudsters=pd.read_csv('fraudsters.csv') #IDs of the known fraudsters (positive class)
users=pd.read_csv('users.csv')
transactions=pd.read_csv('transactions.csv')

## EDA and Feature Engineering

Explore the data to get an understanding of the problem and how to proceed in the later steps. Consider looking at user country, transaction created_date, amount_gbp, type and state. Come up with features that could help you tell fraudsters from other users, as raw data may be of little use to your model.

In [4]:
fraudsters.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 757 entries, 0 to 756
Data columns (total 1 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   USER_ID  757 non-null    object
dtypes: object(1)
memory usage: 6.0+ KB


In [5]:
#Add target column
fraudsters['TARGET'] = 1
fraudsters.head()

,USER_ID,TARGET
0,2c831c76-2d62-41ce-a240-e12f505d389a,1
1,ce2a1146-831e-49a7-aa5f-a3045a2892af,1
2,447abe11-f89a-4819-bea2-e7978b1cf560,1
3,3a186446-c2fb-474b-a8d8-db362643b3d2,1
4,73fa6100-f6f0-4e22-b247-714f4743c125,1


In [6]:
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200250 entries, 0 to 200249
Data columns (total 4 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   ID            200250 non-null  object
 1   CREATED_DATE  200250 non-null  object
 2   COUNTRY       200250 non-null  object
 3   BIRTH_DATE    200250 non-null  object
dtypes: object(4)
memory usage: 6.1+ MB


In [7]:
#Add target column
fraudsters.rename(columns={'USER_ID': 'ID'}, inplace=True)
users=users.merge(fraudsters, on='ID', how='left')
users['TARGET']=users['TARGET'].fillna(0)
users.head()

,ID,CREATED_DATE,COUNTRY,BIRTH_DATE,TARGET
0,46f44852-aaa5-4634-aadd-8cc4eefef3c8,2019-04-22 18:30:30.735,BG,1984-10-22,0.0
1,f17dd8af-2edb-4415-a950-d90a1b5e3e5b,2019-04-15 02:44:24.94,IE,1984-11-04,0.0
2,55e6fcef-f573-4c54-8b27-537adc417e19,2019-04-03 16:10:44.53,PL,1977-09-08,0.0
3,dc03019c-9cf1-4081-a70a-6922a44fe393,2019-04-13 14:16:11.928,FR,1992-09-06,0.0
4,bcd967e5-c273-45a7-a7f5-e7c9e3b19b7e,2019-04-03 15:46:43.997,IE,1993-10-22,0.0


In [8]:
#Check percentage of fraudsters in the dataset
(users['TARGET'].value_counts(normalize=True) * 100).round(2)

TARGET
0.0    99.62
1.0     0.38
Name: proportion, dtype: float64

In [9]:
transactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1068361 entries, 0 to 1068360
Data columns (total 7 columns):
 #   Column        Non-Null Count    Dtype  
---  ------        --------------    -----  
 0   ID            1068361 non-null  object 
 1   USER_ID       1068361 non-null  object 
 2   CREATED_DATE  1068361 non-null  object 
 3   TYPE          1068361 non-null  object 
 4   STATE         1068361 non-null  object 
 5   AMOUNT_GBP    1068361 non-null  float64
 6   CURRENCY      1068361 non-null  object 
dtypes: float64(1), object(6)
memory usage: 57.1+ MB


In [10]:
# Get users data on the main df
df = transactions.merge(users, how='inner', left_on='USER_ID', right_on='ID')
df = df.drop(columns=['ID_y'])
df.rename(columns={'ID_x': 'ID'}, inplace=True)
df.rename(columns={'CREATED_DATE_x'	: 'TRANSACTION_DATE'}, inplace=True)
df.rename(columns={'CREATED_DATE_y'	: 'SIGNUP_DATE'}, inplace=True)
df

,ID,USER_ID,TRANSACTION_DATE,TYPE,STATE,AMOUNT_GBP,CURRENCY,SIGNUP_DATE,COUNTRY,BIRTH_DATE,TARGET
0,f659b44e-cfdf-48de-bcf3-06f47ef26e9f,fd7f3ff6-0ed6-4a85-a7b5-2f205e0ef72f,2019-04-20 18:04:03.93,CARD_PAYMENT,COMPLETED,13.12,PLN,2019-04-18 10:59:26.974,PL,1959-04-08,0.0
1,2ae18b8b-b9bc-4c44-96b1-d43efd8d371d,3979518e-95f7-4b6c-81ae-2f828727d81a,2019-05-03 13:09:57.625,TOPUP,REVERTED,0.01,RON,2019-05-03 13:03:46.883,RO,1993-08-12,0.0
2,0162d352-dd18-40ab-b3ee-cf6584c9a238,75aa5388-9c89-4f72-bc54-67501519585b,2019-04-25 15:37:46.837,TOPUP,COMPLETED,10.00,GBP,2019-04-25 15:31:00.259,GB,1988-04-21,0.0
3,a4e176f7-49ca-462b-9164-2f0645622148,45598164-6362-4ee4-bd70-ffee3bd1d707,2019-04-28 13:52:15.256,EXCHANGE,COMPLETED,0.11,RON,2019-04-28 13:39:18.9,GB,1997-01-13,0.0
4,f6f9135f-fb2b-4a58-bb65-dd9713306a71,5a501ce5-f03c-410d-aabc-434b2cad741d,2019-05-13 16:02:12.081,CARD_PAYMENT,COMPLETED,9.79,EUR,2019-05-03 10:38:55.607,GB,1999-07-22,0.0
...,...,...,...,...,...,...,...,...,...,...,...
1068356,3434ed2e-517b-4452-a199-bf79bd58bbee,b17a0706-0bb1-4b93-9ebc-af10b2d70247,2019-04-27 10:10:04.151,TOPUP,REVERTED,0.01,GBP,2019-04-27 10:07:36.451,GB,1971-02-12,0.0
1068357,383298cb-64d8-4a14-b0c1-3bd2d7695155,de53d6cc-158f-4ca9-98a1-81ee6cfec6f0,2019-04-28 17:56:41.602,FEE,COMPLETED,5.37,EUR,2019-04-27 12:15:18.51,PT,1962-10-31,0.0
1068358,0717b084-2a5d-4214-9062-1b09f085befe,721dfdad-2e4b-4f2b-b296-a9b692f743dc,2019-04-11 12:47:50.575,FEE,COMPLETED,120.41,EUR,2019-04-11 12:34:51.007,AT,1985-07-10,0.0
1068359,145d2404-0fa2-47ce-a6e9-6b0eb0b1041c,bb7d9b60-17b8-4463-be01-e3326e67019b,2019-04-11 10:15:58.585,TOPUP,FAILED,10.00,GBP,2019-04-11 09:56:40.934,GB,2001-04-11,0.0


In [11]:
#Check percentage of fraudulent operations
(df['TARGET'].value_counts(normalize=True) * 100).round(2)

TARGET
0.0    99.23
1.0     0.77
Name: proportion, dtype: float64

In [12]:
#Since one of our models will be Logistic Regression, we need to convert the categorical variables into dummy variables
df = pd.get_dummies(df, columns=['TYPE'])
df = pd.get_dummies(df, columns=['STATE'])
df.head()

,ID,USER_ID,TRANSACTION_DATE,AMOUNT_GBP,CURRENCY,SIGNUP_DATE,COUNTRY,BIRTH_DATE,TARGET,TYPE_ATM,TYPE_CARD_PAYMENT,TYPE_EXCHANGE,TYPE_FEE,TYPE_TOPUP,TYPE_TRANSFER,STATE_COMPLETED,STATE_DECLINED,STATE_FAILED,STATE_REVERTED
0,f659b44e-cfdf-48de-bcf3-06f47ef26e9f,fd7f3ff6-0ed6-4a85-a7b5-2f205e0ef72f,2019-04-20 18:04:03.93,13.12,PLN,2019-04-18 10:59:26.974,PL,1959-04-08,0.0,False,True,False,False,False,False,True,False,False,False
1,2ae18b8b-b9bc-4c44-96b1-d43efd8d371d,3979518e-95f7-4b6c-81ae-2f828727d81a,2019-05-03 13:09:57.625,0.01,RON,2019-05-03 13:03:46.883,RO,1993-08-12,0.0,False,False,False,False,True,False,False,False,False,True
2,0162d352-dd18-40ab-b3ee-cf6584c9a238,75aa5388-9c89-4f72-bc54-67501519585b,2019-04-25 15:37:46.837,10.00,GBP,2019-04-25 15:31:00.259,GB,1988-04-21,0.0,False,False,False,False,True,False,True,False,False,False
3,a4e176f7-49ca-462b-9164-2f0645622148,45598164-6362-4ee4-bd70-ffee3bd1d707,2019-04-28 13:52:15.256,0.11,RON,2019-04-28 13:39:18.9,GB,1997-01-13,0.0,False,False,True,False,False,False,True,False,False,False
4,f6f9135f-fb2b-4a58-bb65-dd9713306a71,5a501ce5-f03c-410d-aabc-434b2cad741d,2019-05-13 16:02:12.081,9.79,EUR,2019-05-03 10:38:55.607,GB,1999-07-22,0.0,False,True,False,False,False,False,True,False,False,False


In [13]:
df['CURRENCY'].unique()

array(['PLN', 'RON', 'GBP', 'EUR', 'HUF', 'BGN', 'CZK', 'SEK', 'USD',
       'SGD', 'CHF', 'HRK', 'NOK', 'JPY', 'CAD', 'UAH', 'DKK', 'TRY',
       'BTC', 'PEN', 'AED', 'AUD', 'BYN', 'MYR', 'RSD', 'SCR', 'IDR',
       'HKD', 'PHP', 'MAD', 'RUB', 'ETH', 'NZD', 'ISK', 'ZAR', 'INR',
       'MXN', 'XPF', 'SAR', 'COP', 'THB', 'ALL', 'TWD', 'ILS', 'GEL',
       'MKD', 'XRP', 'BRL', 'ARS', 'KRW', 'EGP', 'CRC', 'CLP', 'VND',
       'TZS', 'BAM', 'QAR', 'BBD', 'BCH', 'KZT', 'CNY', 'LTC', 'AWG',
       'XCD', 'MUR', 'SRD', 'MDL', 'ZMW', 'BSD', 'BOB', 'GHS', 'DOP',
       'LKR', 'TND', 'MMK', 'KES', 'ETB', 'JOD', 'XOF', 'MOP', 'MGA',
       'MNT', 'OMR', 'CVE', 'GMD', 'LBP', 'MWK', 'BHD', 'NAD', 'BDT',
       'GTQ', 'PKR', 'AMD', 'NGN', 'HNL', 'JMD', 'LAK', 'BWP', 'XAF',
       'MZN', 'AZN', 'FJD', 'BND', 'BZD', 'VES', 'DZD', 'UZS', 'NPR',
       'KGS', 'MVR'], dtype=object)

In [14]:
# Checking nulls in df
df.isnull().sum()

ID                   0
USER_ID              0
TRANSACTION_DATE     0
AMOUNT_GBP           0
CURRENCY             0
SIGNUP_DATE          0
COUNTRY              0
BIRTH_DATE           0
TARGET               0
TYPE_ATM             0
TYPE_CARD_PAYMENT    0
TYPE_EXCHANGE        0
TYPE_FEE             0
TYPE_TOPUP           0
TYPE_TRANSFER        0
STATE_COMPLETED      0
STATE_DECLINED       0
STATE_FAILED         0
STATE_REVERTED       0
dtype: int64

In [15]:
transactions

,ID,USER_ID,CREATED_DATE,TYPE,STATE,AMOUNT_GBP,CURRENCY
0,f659b44e-cfdf-48de-bcf3-06f47ef26e9f,fd7f3ff6-0ed6-4a85-a7b5-2f205e0ef72f,2019-04-20 18:04:03.93,CARD_PAYMENT,COMPLETED,13.12,PLN
1,2ae18b8b-b9bc-4c44-96b1-d43efd8d371d,3979518e-95f7-4b6c-81ae-2f828727d81a,2019-05-03 13:09:57.625,TOPUP,REVERTED,0.01,RON
2,0162d352-dd18-40ab-b3ee-cf6584c9a238,75aa5388-9c89-4f72-bc54-67501519585b,2019-04-25 15:37:46.837,TOPUP,COMPLETED,10.00,GBP
3,a4e176f7-49ca-462b-9164-2f0645622148,45598164-6362-4ee4-bd70-ffee3bd1d707,2019-04-28 13:52:15.256,EXCHANGE,COMPLETED,0.11,RON
4,f6f9135f-fb2b-4a58-bb65-dd9713306a71,5a501ce5-f03c-410d-aabc-434b2cad741d,2019-05-13 16:02:12.081,CARD_PAYMENT,COMPLETED,9.79,EUR
...,...,...,...,...,...,...,...
1068356,3434ed2e-517b-4452-a199-bf79bd58bbee,b17a0706-0bb1-4b93-9ebc-af10b2d70247,2019-04-27 10:10:04.151,TOPUP,REVERTED,0.01,GBP
1068357,383298cb-64d8-4a14-b0c1-3bd2d7695155,de53d6cc-158f-4ca9-98a1-81ee6cfec6f0,2019-04-28 17:56:41.602,FEE,COMPLETED,5.37,EUR
1068358,0717b084-2a5d-4214-9062-1b09f085befe,721dfdad-2e4b-4f2b-b296-a9b692f743dc,2019-04-11 12:47:50.575,FEE,COMPLETED,120.41,EUR
1068359,145d2404-0fa2-47ce-a6e9-6b0eb0b1041c,bb7d9b60-17b8-4463-be01-e3326e67019b,2019-04-11 10:15:58.585,TOPUP,FAILED,10.00,GBP
